# Final Assignment, Coursera IBM Data Science Capstone

## This notebook will be used to create and share all requirements from the last course

### 1) First assignment : Import librairies

In [1]:
!pip install numpy --upgrade
!pip install pandas
import pandas as pd
import numpy as np

     |████████████████████████████████| 15.3 MB 17.7 MB/s eta 0:00:01
ERROR: tensorflow 2.1.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.5.0 which is incompatible.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### 2) Second assignment : Let's go Toronto

#### a)  Part 1 - Scrapping Wikipedia

##### Import BS and requests, define Next Step to automatically stop in case of error

In [3]:
from bs4 import BeautifulSoup
import requests
next_step = True

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url)

##### Use BeautifulSoup to parse url

In [5]:
soup = BeautifulSoup(html_data.text, 'html.parser')

##### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [6]:
toronto_neighborhoods_source = pd.DataFrame(columns=["PostalCode", "Borough", "Neighbourhood"])

##### Get table from url. Wikipedia uses the class "wikitable sortable" to store the data we need

In [7]:
table = soup.find("table", { "class" : "wikitable sortable" }).find_all("tr")

##### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [8]:
for row in table:
    col = row.find_all("td")
    if len(col) > 0:
        cells = row.find_all("td")
        PostalCode = cells[0].get_text(strip=True) #strip = True removes "\n" at the end of each text value
        Borough = cells[1].get_text(strip=True)
        Neighbourhood = cells[2].get_text(strip=True)
        if(Borough != "Not assigned"): #we remove "Not assigned" boroughs
            toronto_neighborhoods_source = toronto_neighborhoods_source.append({"PostalCode":PostalCode, "Borough":Borough, "Neighbourhood":Neighbourhood}, ignore_index=True)


##### Check dataframe head

In [9]:
toronto_neighborhoods_source.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [10]:
if (sum(toronto_neighborhoods_source.groupby(['PostalCode']).count()["Borough"]) == toronto_neighborhoods_source["PostalCode"].count()) & next_step:
    print("Already combined, go next step")
else:
    print("Something to do")
    next_step = False
#Seems to be done in Wikipedia table, count of rows grouped by PostalCode = count of rows without groups

Already combined, go next step


##### If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [11]:
if (toronto_neighborhoods_source[toronto_neighborhoods_source["Neighbourhood"] == "Not assigned"].count()["PostalCode"]) == 0 & next_step:
    print("No not assigned neighborhood")
else:
    print("Something to do")
    next_step = False
#No "Not assigned" neighborhood with "Assigned" borough

No not assigned neighborhood


##### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [12]:
print(toronto_neighborhoods_source.shape)

(103, 3)


#### b) Part 2 - We now need to get the latitude and the longitude coordinates of each neighborhood

##### Import geocoder

In [13]:
!pip install geocoder
import geocoder # import geocoder
import json

     |████████████████████████████████| 98 kB 9.2 MB/s  eta 0:00:01


##### Start to retrieve the coordinates using geocoder with Bing api

In [14]:
# initialize your variable to None
location = None
lat_long_df = pd.DataFrame(columns = ["PostalCode","Latitude", "Longitude"])
lim = 0

nbPostalCode = toronto_neighborhoods_source.shape[0]

for i in range(nbPostalCode):
    # loop until you get the coordinates of each postal code in the dataframe
    PostalCode = toronto_neighborhoods_source.iloc[i]["PostalCode"]
    address = ('{}, Toronto, Ontario'.format(PostalCode))
    
    location = geocoder.bing(address, key="AleEpOyasZ0iz0IAQvg1-IlnUBkCFWNNZxogPQ7azEB3Ztt8MOzf3P7X2Unxbcm8")
    result = json.dumps(location.json)
    data = json.loads(result)

    if(data["status"] == "OK"):
        Latitude = data["lat"]
        Longitude = data["lng"]
        print(address, " OK")
        lat_long_df = lat_long_df.append({"PostalCode":PostalCode, "Latitude":Latitude, "Longitude":Longitude}, ignore_index=True)
    else:
        Latitude = 0
        Longitude = 0
        print(address, " KO")


M3A, Toronto, Ontario  OK
M4A, Toronto, Ontario  OK
M5A, Toronto, Ontario  OK
M6A, Toronto, Ontario  OK
M7A, Toronto, Ontario  OK
M9A, Toronto, Ontario  OK
M1B, Toronto, Ontario  OK
M3B, Toronto, Ontario  OK
M4B, Toronto, Ontario  OK
M5B, Toronto, Ontario  OK
M6B, Toronto, Ontario  OK
M9B, Toronto, Ontario  OK
M1C, Toronto, Ontario  OK
M3C, Toronto, Ontario  OK
M4C, Toronto, Ontario  OK
M5C, Toronto, Ontario  OK
M6C, Toronto, Ontario  OK
M9C, Toronto, Ontario  OK
M1E, Toronto, Ontario  OK
M4E, Toronto, Ontario  OK
M5E, Toronto, Ontario  OK
M6E, Toronto, Ontario  OK
M1G, Toronto, Ontario  OK
M4G, Toronto, Ontario  OK
M5G, Toronto, Ontario  OK
M6G, Toronto, Ontario  OK
M1H, Toronto, Ontario  OK
M2H, Toronto, Ontario  OK
M3H, Toronto, Ontario  OK
M4H, Toronto, Ontario  OK
M5H, Toronto, Ontario  OK
M6H, Toronto, Ontario  OK
M1J, Toronto, Ontario  OK
M2J, Toronto, Ontario  OK
M3J, Toronto, Ontario  OK
M4J, Toronto, Ontario  OK
M5J, Toronto, Ontario  OK
M6J, Toronto, Ontario  OK
M1K, Toronto

##### Check data in json with coordinates

In [15]:
print(lat_long_df)

    PostalCode   Latitude  Longitude
0          M3A  43.756123 -79.329636
1          M4A  43.726780 -79.310738
2          M5A  43.655354 -79.365044
3          M6A  43.721996 -79.445915
4          M7A  43.663910 -79.388733
..         ...        ...        ...
98         M8X  43.652699 -79.511276
99         M4Y  43.666286 -79.382446
100        M7Y  43.651894 -79.381714
101        M8Y  43.633709 -79.496521
102        M8Z  43.629711 -79.517479

[103 rows x 3 columns]


##### Check coordinates dataframe size vs wikipedia dataframe size

In [16]:
if(lat_long_df.shape[0] == toronto_neighborhoods_source.shape[0]) & next_step:
    print("Merge is possible, data can be slightely differents because of Bing geocoder")
else:
    print("Need to import csv")
    lat_long_df = pd.read_csv("https://cocl.us/Geospatial_data")

Merge is possible, data can be slightely differents because of Bing geocoder


##### Finally, merge coordinates with wikipedia data

In [17]:
toronto_with_lat_long = pd.merge(toronto_neighborhoods_source, lat_long_df, on=['PostalCode'])
toronto_with_lat_long.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.756123,-79.329636
1,M4A,North York,Victoria Village,43.726780,-79.310738
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655354,-79.365044
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.721996,-79.445915
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.663910,-79.388733


#### c) Part 3 - Explore and cluster the neighborhoods in Toronto

##### Import depedencies

In [18]:
!pip install folium --upgrade
import folium

     |████████████████████████████████| 94 kB 6.4 MB/s  eta 0:00:01


##### Let's start by the map of neighbourhood

In [19]:
# create map of Toronto using latitude and longitude values
address = ('Toronto, Ontario')
location = geocoder.bing(address, key="AleEpOyasZ0iz0IAQvg1-IlnUBkCFWNNZxogPQ7azEB3Ztt8MOzf3P7X2Unxbcm8")
result = json.dumps(location.json)
data = json.loads(result)

if(data["status"] == "OK"):
    Latitude = data["lat"]
    Longitude = data["lng"]
    print(address, " OK")

map_toronto = folium.Map(location=[Latitude, Longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_with_lat_long['Latitude'], toronto_with_lat_long['Longitude'], toronto_with_lat_long['Borough'], toronto_with_lat_long['Neighbourhood']):
    html = "<b>" + borough + "</b>" + "<br>" + "<i>" + neighbourhood.replace(",", "\n") + "</i>"
    iframe = folium.IFrame(html=html, width=200, height=100)
    popup = folium.Popup(iframe, max_width=2650)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Toronto, Ontario  OK


##### Next, we need to retrieve Foursquare venues in each borough (200 max per borough)

In [ ]:
toronto_venues = pd.DataFrame(columns = ["PostalCode", "Venue", "VenueCategory"])

ACCESS_TOKEN = 'CXLOL41WLZQXOQVTXHRLMNIIMQW11IW1MFM0ALPAGV3PZDGK'
RADIUS = 500
LIMIT = 200
VERSION = '20180605'

for postal, latitude, longitude in zip(toronto_with_lat_long["PostalCode"], toronto_with_lat_long["Latitude"], toronto_with_lat_long["Longitude"]):
    url = 'https://api.foursquare.com/v2/venues/explore?oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(ACCESS_TOKEN, latitude, longitude, VERSION, RADIUS, LIMIT)
    print(url)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    for res in results:
        venue = res['venue']['name']
        category = res['venue']['categories'][0]['name']
        toronto_venues = toronto_venues.append({"PostalCode":postal, "Venue":venue, "VenueCategory":category}, ignore_index=True)

https://api.foursquare.com/v2/venues/explore?oauth_token=CXLOL41WLZQXOQVTXHRLMNIIMQW11IW1MFM0ALPAGV3PZDGK&ll=43.75612258911133,-79.32963562011719&v=20180605&radius=500&limit=200
https://api.foursquare.com/v2/venues/explore?oauth_token=CXLOL41WLZQXOQVTXHRLMNIIMQW11IW1MFM0ALPAGV3PZDGK&ll=43.72677993774414,-79.31073760986328&v=20180605&radius=500&limit=200
https://api.foursquare.com/v2/venues/explore?oauth_token=CXLOL41WLZQXOQVTXHRLMNIIMQW11IW1MFM0ALPAGV3PZDGK&ll=43.65535354614258,-79.36504364013672&v=20180605&radius=500&limit=200
https://api.foursquare.com/v2/venues/explore?oauth_token=CXLOL41WLZQXOQVTXHRLMNIIMQW11IW1MFM0ALPAGV3PZDGK&ll=43.72199630737305,-79.44591522216797&v=20180605&radius=500&limit=200
https://api.foursquare.com/v2/venues/explore?oauth_token=CXLOL41WLZQXOQVTXHRLMNIIMQW11IW1MFM0ALPAGV3PZDGK&ll=43.663909912109375,-79.38873291015625&v=20180605&radius=500&limit=200
https://api.foursquare.com/v2/venues/explore?oauth_token=CXLOL41WLZQXOQVTXHRLMNIIMQW11IW1MFM0ALPAGV3PZDGK&ll=

##### Let's count venues by neighbourhood

In [ ]:
toronto_venues.groupby('PostalCode').count()

##### Find out how many unique categories

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['VenueCategory'].unique())))

##### One hot encoding (transforms categories to dummy)

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
#toronto_onehot[toronto_onehot['PostalCode'] == 'Marble Hill']

##### Size of this onehot dataframe

In [ ]:
toronto_onehot.shape

##### Group by neighbourhood and mean of frequency

In [ ]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped

##### Check the shape

In [ ]:
toronto_grouped.shape

##### Function to return most common venues

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

##### Cluster neighbourhood

In [ ]:
toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

toronto_grouped_clustering

##### Import KMeans

In [ ]:
!pip install sklearn
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline

##### Find best cluster number

In [ ]:
# run k-means clustering
distortions = []
K = range(1,30)
kclusters = 5
for kclusters in K:
    kmeans = KMeans(n_clusters=kclusters, random_state=0, init='k-means++').fit(toronto_grouped_clustering)
    distortions.append(kmeans.inertia_)

plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

##### The best K seems to be 8 ?

In [ ]:
kclusters = 8
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_

##### Let's merge all those data

In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_final = pd.merge(neighborhoods_venues_sorted, lat_long_df, on=['PostalCode'])
toronto_final = pd.merge(toronto_final, toronto_neighborhoods_source, on=['PostalCode'])
toronto_final.head()

##### Import matplotlib colors and modules

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

##### And finally, create the map

In [ ]:
# create map
# create map of Toronto using latitude and longitude values
address = ('Toronto, Ontario')
location = geocoder.bing(address, key="AleEpOyasZ0iz0IAQvg1-IlnUBkCFWNNZxogPQ7azEB3Ztt8MOzf3P7X2Unxbcm8")
result = json.dumps(location.json)
data = json.loads(result)

if(data["status"] == "OK"):
    Latitude = data["lat"]
    Longitude = data["lng"]
    print(address, " OK")
    
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, borough, cluster in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Neighbourhood'], toronto_final['Borough'], toronto_final['Cluster Labels']):
    #label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    html = "<b>" + borough + "</b>" + "<br>" + "<i>" + neighbourhood.replace(",", "\n") + "</i>" + "</br>" + "Cluster : " + str(cluster)
    iframe = folium.IFrame(html=html, width=200, height=100)
    popup = folium.Popup(iframe, max_width=2650)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=popup,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3) Third assignment, week 1 : Should I stay or should I go ?

#### - Introduction - where you discuss the business problem and who would be interested in this project

Moving out can be a difficult decision. If you really love the borough you live in, it can be heartbreaker to move out to another borough in another city / state / country. When my company asked me if I was interested in moving out from Lille to Paris, I asked myself if it was a good idea. The main goal of this project is to try to automatically understand what makes your place so special for you, and which borough in a given city could fit with your requirements. So, I'd like to create a tool in which you enter your current address and the city you're wondering about moving out, and which will try to find the best borough to you.

#### - Data - where you describe the data that will be used to solve the problem and the source of the data

##### To be able to make this tool operationnal, we need 4 inputs : 
 - your current location (complete address would be great)
 - the radius in which your favorite venues are located (in kilometers)
 - the city, state, country you're moving out 
 - a radius of distance from this city (you probably won't live at the exact center of the city given by an API) (in kilometers)

With those data, the tool will call Bing Map API to get the latitude and longitude of your current location, then it will find the latitude and longitude of the city you're moving out.
Bing Map API returns a json with a lot of data including latitude and longitude of a given address or city. We'll then create virtual boroughs around the center of the city you're moving out thanks to a grid tool : 

![Image of grid](https://reivaxweb.me/Coursera/grid_on_map.jpg)

With the explore function of the foursquare API, the tool will find all venues in the radius you defined around your place, and will then do the same for each virtual borough generated by the grid tool. This API gets catergory of hundreds of venues, and those categories will help to find common data between your current address and the new place. A clustering unsupervised algorythm will then try to find virtual boroughs that match with your current address in the new city.


### 3) Third assignment, week 2 : Should I stay or should I go ?

#### - Methodology section - which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why
##### I'll use a kmean approach. I choosed to live in the center of Lille, so I was obviously interested by all venues around the town center of this city, and I wanted to find the same venues in Paris. First, I used Foursquare API to get all venues in a 1 km radius because it's a good distance to me to walk to my favorites venues, then I grouped the category of those venues by borough and created clusters based on those data.

#### - Results section - where you discuss the results
##### The result section is the map with colored clusters and the top ten venues in each borough

#### - Discussion section - where you discuss any observations you noted and any recommendations you can make based on the results
##### At the end of the notebook, after the clustered map

#### - Conclusion section - where you conclude the report
##### At the end of the notebook, after the discussion

##### Let's find Latitude and Longitude of Lille and of each borough in Paris

In [ ]:
url = "https://fr.wikipedia.org/wiki/Arrondissements_de_Paris"
html_data = requests.get(url)

In [ ]:
soup = BeautifulSoup(html_data.text, 'html.parser')

In [ ]:
paris_neighborhoods_source = pd.DataFrame(columns=["PostalCode"])

In [ ]:
table = soup.find("table", { "class" : "wikitable sortable alternance" }).find_all("tr")

In [ ]:
for row in table:
    col = row.find_all("td")
    if len(col) > 0:
        cells = row.find_all("td")
        PostalCode = cells[0].get_text(strip=True) #strip = True removes "\n" at the end of each text value
        paris_neighborhoods_source = paris_neighborhoods_source.append({"PostalCode":PostalCode}, ignore_index=True)

In [ ]:
paris_neighborhoods_source = paris_neighborhoods_source.head(20)

In [ ]:
paris_neighborhoods_source.shape

In [ ]:
paris_neighborhoods_source.head(20)

In [ ]:
# initialize your variable to None
location = None
lille_paris_with_lat_long = pd.DataFrame(columns = ["PostalCode","Latitude", "Longitude"])
lim = 0

nbPostalCode = paris_neighborhoods_source.shape[0]

for i in range(nbPostalCode):
    # loop until you get the coordinates of each postal code in the dataframe
    PostalCode = paris_neighborhoods_source.iloc[i]["PostalCode"]
    address = ('{} arrondissement, Paris, France'.format(PostalCode))
    
    location = geocoder.bing(address, key="AleEpOyasZ0iz0IAQvg1-IlnUBkCFWNNZxogPQ7azEB3Ztt8MOzf3P7X2Unxbcm8")
    result = json.dumps(location.json)
    data = json.loads(result)

    if(data["status"] == "OK"):
        Latitude = data["lat"]
        Longitude = data["lng"]
        print(address, " OK")
        lille_paris_with_lat_long = lille_paris_with_lat_long.append({"PostalCode":PostalCode, "Latitude":Latitude, "Longitude":Longitude}, ignore_index=True)
    else:
        Latitude = 0
        Longitude = 0
        print(address, " KO")

##### Let's add Lille to this dataframe

In [ ]:
PostalCode = "Lille"
address = ('{}, France'.format(PostalCode))
location = geocoder.bing(address, key="AleEpOyasZ0iz0IAQvg1-IlnUBkCFWNNZxogPQ7azEB3Ztt8MOzf3P7X2Unxbcm8")
result = json.dumps(location.json)
data = json.loads(result)

if(data["status"] == "OK"):
    Latitude = data["lat"]
    Longitude = data["lng"]
    print(address, " OK")
    lille_paris_with_lat_long = lille_paris_with_lat_long.append({"PostalCode":PostalCode, "Latitude":Latitude, "Longitude":Longitude}, ignore_index=True)
else:
    Latitude = 0
    Longitude = 0
    print(address, " KO")

##### Let's check dataframe

In [ ]:
lille_paris_with_lat_long

##### Mapping Lille and all boroughs in Paris

In [ ]:
# create map of Toronto using latitude and longitude values
address = ('France')
location = geocoder.bing(address, key="AleEpOyasZ0iz0IAQvg1-IlnUBkCFWNNZxogPQ7azEB3Ztt8MOzf3P7X2Unxbcm8")
result = json.dumps(location.json)
data = json.loads(result)

if(data["status"] == "OK"):
    Latitude = data["lat"]
    Longitude = data["lng"]
    print(address, " OK")

map_france = folium.Map(location=[Latitude, Longitude], zoom_start=6)

# add markers to map
for lat, lng, postalcode in zip(lille_paris_with_lat_long['Latitude'], lille_paris_with_lat_long['Longitude'], lille_paris_with_lat_long['PostalCode']):
    html = "<b>" + postalcode + "</b>"
    iframe = folium.IFrame(html=html, width=200, height=100)
    popup = folium.Popup(iframe, max_width=2650)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_france)  
    
map_france

##### Let's use Foursquare API to get all venues in a radius of 1000 meters around each cluster

In [ ]:
lille_paris_venues = pd.DataFrame(columns = ["PostalCode", "Venue", "VenueCategory"])

ACCESS_TOKEN = 'CXLOL41WLZQXOQVTXHRLMNIIMQW11IW1MFM0ALPAGV3PZDGK'
RADIUS = 1000
LIMIT = 100
VERSION = '20180605'

for postal, latitude, longitude in zip(lille_paris_with_lat_long["PostalCode"], lille_paris_with_lat_long["Latitude"], lille_paris_with_lat_long["Longitude"]):
    url = 'https://api.foursquare.com/v2/venues/explore?oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(ACCESS_TOKEN, latitude, longitude, VERSION, RADIUS, LIMIT)
    print(url)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    for res in results:
        venue = res['venue']['name']
        category = res['venue']['categories'][0]['name']
        lille_paris_venues = lille_paris_venues.append({"PostalCode":postal, "Venue":venue, "VenueCategory":category}, ignore_index=True)

In [ ]:
lille_paris_venues

##### Let's count venue by place

In [ ]:
lille_paris_venues.groupby('PostalCode').count()

##### Find out how many unique categories

In [ ]:
print('There are {} uniques categories.'.format(len(lille_paris_venues['VenueCategory'].unique())))

##### One hot encoding (transforms categories to dummy)

In [ ]:
# one hot encoding
lille_paris_onehot = pd.get_dummies(lille_paris_venues[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lille_paris_onehot['PostalCode'] = lille_paris_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [lille_paris_onehot.columns[-1]] + list(lille_paris_onehot.columns[:-1])
lille_paris_onehot = lille_paris_onehot[fixed_columns]

lille_paris_onehot.head()

##### Size of this onehot dataframe

In [ ]:
lille_paris_onehot.shape

##### Group by neighbourhood and mean of frequency

In [ ]:
lille_paris_grouped = lille_paris_onehot.groupby('PostalCode').mean().reset_index()
lille_paris_grouped

##### Function to return most common venues

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Now let's create the new dataframe and display the top 10 venues for each neighborhood, to have a quick overview of the data

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = lille_paris_grouped['PostalCode']

for ind in np.arange(lille_paris_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lille_paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

##### Cluster neighbourhood (remove PostalCode as it must not influence the kmean)

In [ ]:
lille_paris_grouped_clustering = lille_paris_grouped.drop('PostalCode', 1)
lille_paris_grouped_clustering

##### Create clusters with those data

In [ ]:
# run k-means clustering
distortions = []
K = range(1,10)

for kclusters in K:
    kmeans = KMeans(n_clusters=kclusters, random_state=0, init='k-means++').fit(lille_paris_grouped_clustering)
    distortions.append(kmeans.inertia_)

plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

##### Let's use a 3 clusters kmean

In [ ]:
kclusters = 3
kmeans = KMeans(n_clusters=kclusters, random_state=0, init='k-means++').fit(lille_paris_grouped_clustering)

kmeans.labels_

##### Let's merge all those data

In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

In [ ]:
lille_paris_final = pd.merge(neighborhoods_venues_sorted, lat_long_df, on=['PostalCode'])
lille_paris_final.head()

##### And finally, we can create the map

In [ ]:
# create map
# create map of France using latitude and longitude values
address = ('France')
location = geocoder.bing(address, key="AleEpOyasZ0iz0IAQvg1-IlnUBkCFWNNZxogPQ7azEB3Ztt8MOzf3P7X2Unxbcm8")
result = json.dumps(location.json)
data = json.loads(result)

if(data["status"] == "OK"):
    Latitude = data["lat"]
    Longitude = data["lng"]
    print(address, " OK")
    
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, postalcode, cluster in zip(lille_paris_final['Latitude'], lille_paris_final['Longitude'], lille_paris_final['PostalCode'], lille_paris_final['Cluster Labels']):
    #label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    html = "<b>" + postalcode + "</b>" + "</br>" + "Cluster : " + str(cluster)
    iframe = folium.IFrame(html=html, width=200, height=100)
    popup = folium.Popup(iframe, max_width=2650)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=popup,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Discussion : to improve the model, I could have put my real address instead of Lille, to be more accurate. If I wanted to really use this tool, I would have split Paris in squares of 1 km² sliding each 200m for instance, because here we only have the center of each borough. 

### Conclusion : the tool is really accurate, the chosen boroughs in Paris that seems most similar to Lille are 18th, 19th, 20th and 11th, and my real choice was to move out to 11th !